# Create Hagmann connectome with Brodmann areas

Hagmann presented a connectome in [Mapping the Structural Core of Human Cerebral Cortex](https://doi.org/10.1371/journal.pbio.0060159) and published the data. This notebook takes the published data, produces a [NetworkX graph](https://networkx.github.io/documentation/stable/index.html), and writes it to disk in [gpickle](https://docs.python.org/3/library/pickle.html) format.

1. Download Hagmann connectome from TheVirtualBrain github
2. Download Brodmann areas volume from MRIcron github
3. Load data
4. Build a graph and write it to disk.

## Download Hagmann connectome from TheVirtualBrain github

In [1]:
import requests

url = 'https://github.com/the-virtual-brain/tvb-data/raw/master/tvb_data/connectivity/connectivity_998.zip'

r = requests.get(url)  
with open('./connectivity_998.zip', 'wb') as f:
    f.write(r.content)

In [2]:
from shutil import unpack_archive
unpack_archive('./connectivity_998.zip', 'connectivity_998/')

## Download Brodmann areas volume from MRIcron github

In [3]:
import nibabel as nib
url = "https://github.com/neurolabusc/MRIcron/raw/master/niftiview7/templates/brodmann.nii.gz"
brodmann_filename = url.rsplit('/', 1)[1]
r = requests.get(url)
open(brodmann_filename, 'wb').write(r.content)



174606

## Load data

In [5]:
import numpy as np

# Load broadmann volume and compute inverse transform
brodmann_nii = nib.load(brodmann_filename)
inv_transform = np.linalg.inv(brodmann_nii.affine)

# read nodes region names and centers
centres = np.genfromtxt('connectivity_998/centres.txt',
                        dtype='|U256,i,i,i',
#                        encoding='utf-8',
                        names=['name','x','y','z'])

# Read labels
labels = np.genfromtxt('connectivity_998/raw_order_labels.txt',
                        dtype='|U256').tolist()

# It seems there are inconsistencies in region names, between centres and labels. Fix it
ind = centres['name'] == 'rBSTS'
centres['name'][ind] = 'rBTST'
ind = centres['name'] == 'lBSTS'
centres['name'][ind] = 'lBTST'


# convert nodes region names to dict
centres_list = list()
positions_list = list()
for idx, r in enumerate(centres):
    centres_list.append(r['name'])
    positions_list.append([r['x'], r['y'],r['z']])
    
    
weights = np.genfromtxt('connectivity_998/weights.txt')


## Build a graph and write it to disk.

In [6]:
import networkx as nx

# Create a graph from weighted adjacency matrix
weights_graph = nx.from_numpy_matrix(weights)

# Create a graph from nodes' positions
graph = nx.Graph()
graph.add_nodes_from(range(0,len(positions_list)))

for n, p in enumerate(positions_list):
    graph.node[n]['x'] = p[0]
    graph.node[n]['y'] = p[1]
    graph.node[n]['z'] = p[2]
    
    point_in_volume = [int(p[0]+inv_transform[0,3]),
                       int(p[1]+inv_transform[1,3]),
                       int(p[2]+inv_transform[2,3])]
    
    graph.node[n]['brodmann_area'] = brodmann_nii.get_data()[point_in_volume[0],
                                                  point_in_volume[1],
                                                  point_in_volume[2],]
    
# Copy weighted edges
graph.add_weighted_edges_from(weights_graph.edges(data='weight'))

# Write to disk
nx.write_gpickle(graph,'Hagmann_998.gpickle')

# Generate some signals

In [2]:
import mne
import numpy as np

In [26]:
signals = np.tile(np.linspace(0,12000-1,120000), (998,1))
idx = int(998/2)
signals[0:idx,:] = signals[0:idx,:]+np.pi 
print(signals.shape)

(998, 120000)


In [27]:
signals = np.sin(signals)

In [28]:
%matplotlib inline
info = mne.create_info(998, 1000, 'eeg')
raw = mne.io.RawArray(signals, info)
#raw.plot()

Creating RawArray with float64 data, n_channels=998, n_times=120000
    Range : 0 ... 119999 =      0.000 ...   119.999 secs
Ready.


In [29]:
raw.save('sub-mni_sim-simplehalf.fif')

This filename (sub-mni_sim-simplehalf.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Writing /Users/spherik/Documents/DevLibs/python_for_brain/hagmann_connetome/sub-mni_sim-simplehalf.fif


<ipython-input-29-0c772cea1daf>:1: RuntimeWarning: This filename (sub-mni_sim-simplehalf.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw.save('sub-mni_sim-simplehalf.fif')


Closing /Users/spherik/Documents/DevLibs/python_for_brain/hagmann_connetome/sub-mni_sim-simplehalf.fif [done]
